Installation:

```
conda create -n gaiafhds python=3.13
conda activate gaiafhds
conda install -c conda-forge numpy astropy matplotlib pyvo ipympl ipykernel ipyaladin
```

In [ ]:
import warnings

import numpy as np

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

import pyvo

import astropy
from astropy import units as u
#import astropy.table
#import astropy.visualization
#import astropy.coordinates

from ipyaladin import Aladin

import urllib.request
import pathlib
import pickle


In [ ]:
sternhaufen_name = "M11"

In [ ]:
from ipyaladin import __version__, __aladin_lite_version__
print("version:", __version__, "running Aladin Lite:", __aladin_lite_version__)

In [ ]:
aladin = Aladin(target=sternhaufen_name, fov=3.0, survey="P/DSS2/color", height=600)
aladin

In [ ]:

# Die Himmelskoordinaten
# Mit Hilfe von astropy können wir diese direkt herausfinden!
sternhaufen_position = astropy.coordinates.SkyCoord.from_name(sternhaufen_name)
dec_grad = sternhaufen_position.dec.to_value(u.deg)
ra_grad = sternhaufen_position.ra.to_value(u.deg)

# Der Radius, innerhalb dessens wir den Katalog laden wollen:
radius_grad = 0.5 # Dieser Radius ist in Grad angegeben.
radius_grad = 0.1 # Dieser Radius ist in Grad angegeben.


print("Die Koordinaten (in Grad): DEC = ", dec_grad, ", und RA = ", ra_grad)
# Du könntest das mit dem vergleichten, was Wikipedia für M4 angibt.

# Jetzt geben wir an, woher wir die Daten laden wollen:
tap_service = pyvo.dal.TAPService("https://gaia.ari.uni-heidelberg.de/tap")

# Und wir beschreiben, in einer speziellen Sprache, welche Daten wir wollen.
# In den ersten Zeilen, nach SELECT, geben wir die namen der Spalten an,
# die wir brauchen.
# Dann folgt die Angabe der Himmelsregion um den Sternhaufen.
tap_anfrage = """
SELECT ra, dec, pmra, pmdec,
phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, parallax, parallax_error
FROM gaiadr2.gaia_source
WHERE 1 = CONTAINS(POINT('ICRS', ra, dec),
                   CIRCLE('ICRS', {}, {}, {}))
AND phot_g_mean_mag < 17.0
""".format(ra_grad, dec_grad, radius_grad)
print("Die Anfrage an der Server lautet:")
print(tap_anfrage)

# Nun stellen wir diese Anfrage, und bekommen (wenn alles klappt) die Daten.
print("Die Anfrage wird nun gestellt.")
print("Es kann einen Moment dauern, bis die Daten angekommen sind.")
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", module='astropy.*')
    tap_daten = tap_service.run_sync(tap_anfrage)

katalog = tap_daten.to_table()

print("Wir haben den Katalog bekommen. Zusammenfassung:")
print(katalog.info)
# Diese Zusammenfassung ist nützlich, daher schreiben wir sie in eine Datei:
with open("info_katalog.txt", "w") as datei:
    datei.write(str(katalog.info))

# Jetzt schreiben wir den Katalog in eine einfache Text-Datei,
# als eine große Tabelle, im Format "tab-separated values", oft tsv genannt.
katalog.write(f"katalog_um_{sternhaufen_name}.tsv", format="ascii.tab", overwrite=True)
print("Fertig!")

In [ ]:
aladin.add_table(
    katalog,
    shape="circle",
    color="red",
    ra_field="ra",
    dec_field="dec",
)

In [ ]:


# Wir laden den Katalog so wie in der Einleitung gezeigt:
katalog = np.genfromtxt(f"katalog_um_{sternhaufen_name}.tsv", delimiter="\t", names=True)

# Und erstellen ein Diagram mit Matplotlib, folgendes wird gleich besprochen.
plt.figure(figsize=(6, 6))
plt.plot(katalog["pmra"], katalog["pmdec"],
    marker=".", markersize=1, linestyle="None")

plt.title(f"Gaia-Eigenbewegungen im Himmelsauschnitt um {sternhaufen_name}")
plt.xlabel("Eigenbewegung in RA [welche Einheit?]")
plt.ylabel("Eigenbewegung in Dec [welche Einheit?]")

plt.tight_layout()
plt.show()

In [ ]:

#katalog = np.genfromtxt(f"katalog_{sternhaufen_name}.tsv", delimiter="\t", names=True)
katalog = np.genfromtxt(f"katalog_um_{sternhaufen_name}.tsv", delimiter="\t", names=True)
#katalog = np.delete(katalog, 0) # Entfernen der nan-Zeile, die durch die
# Striche unter den Spaltennamen ensteht.

# Als erstes geben wir mal die Namen der Spalten aus:
print("Namen der Spalten:", katalog.dtype.names)
print("Anzahl der Sterne:", len(katalog))

plt.figure()
plt.plot(
    katalog["phot_bp_mean_mag"] - katalog["phot_rp_mean_mag"], # Farbe: B - R
    katalog["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    label="M4",
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
    )

# Wie schon im Modul 2 gemacht ändern wir die Richtung der y-Achse:
plt.gca().invert_yaxis()

# Und Beschriften das Diagram:
plt.title(f"Gaia-FHD von {sternhaufen_name}")
plt.xlabel("Farbe B - R [mag]")
plt.ylabel("Scheinbare Helligkeit R [mag]")
plt.legend()

# Optional: setzte folgende Zeile direkt vor show() oder savefig(),
# dann wird der Abstand zwischen Beschrifung und Diagram optimiert:
plt.tight_layout()

# Und zum Schluss eines von
plt.show()


In [ ]:
# Download the file containing all the isochrones (about 48 MB):

local_isochrones_filepath = pathlib.Path("MIST_Gaia_isochrones.pickle")

isochrones_url = ""
if not local_isochrones_filepath.exists():
    urllib.request.urlretrieve(isochrones_url, local_isochrones_filepath)

# Read this file with the following code (see below for explanations):
with open(local_isochrones_filepath, 'rb') as f:
    (isochrones, vrots, metallicities, logages) = pickle.load(f)

In [ ]:
def extinction_law(A_V):
    """
    Returns extinction values for the different Gaia filters, based on some reference total extinction A_V

    The particular values we use here are from Wang and Chen (2019) ApJ 877 116:
    https://iopscience.iop.org/article/10.3847/1538-4357/ab1c61
    """
    #return {"g": ref_ext*3.30, "r": ref_ext*2.31, "i":ref_ext*1.71 } # Table 2 of Yuan et al. (2013) https://doi.org/10.1093/mnras/stt039
    return {"G": A_V*0.8, "RP": A_V*0.589, "BP":A_V*1.002 } # Table 3 of Wang and Chen (2019)



def mag_absolute_to_apparent(m, distance, extinction):
    """
    Function that corrects absolute magnitudes by distance modulus and extinction, to get apparent magnitudes
    
    m: absolute magnitude(s)
    distance: in pc
    extinction: in mag
    """
    return m + 5.0*np.log10(distance) - 5 + extinction 


def get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance=10, A_V=0):
    """
    Helper function to get a specific isochrone at a specific distance and total extinction
    """
    
    # Computing the extinctions:
    extinctions = extinction_law(A_V)
    
    # Obtaining the corrected magnitudes:
    iso_app_g = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["G"], distance, extinctions["G"])
    iso_app_r = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["BP"], distance, extinctions["BP"])
    iso_app_b = mag_absolute_to_apparent(isochrones[vrot_index][metallicity_index][age_index]["RP"], distance, extinctions["RP"])
    
    # A fancy string describing the isochrone, with some LaTeX formatting:
    label_string = f"Age {(10.0**logages[age_index])/(1e9):.2f} Gyr, " \
        + f"[Fe/H] = {metallicities[metallicity_index]:.2f}, " \
        + r"$v/v_{\mathrm{crit}}$ = " + f"{vrots[vrot_index]}, " \
        + f"$d$ = {distance:.0f} pc, " \
        + r"$A_{\mathrm{V}}$ = " + f"{A_V:.2f}"

    return (iso_app_g, iso_app_r, iso_app_b, label_string)



In [ ]:
# Initial position of sliders:
init_distance = 10.0
init_age_index = 50
init_metallicity_index = 12
init_A_V = 0.0
init_vrot_index = 0

fig, ax = plt.subplots(figsize=(10, 6))
fig.subplots_adjust(left=0.30, bottom=0.1) # making room for the sliders

# Plotting the observed data
cbar = ax.plot(
    katalog["phot_bp_mean_mag"] - katalog["phot_rp_mean_mag"], # Farbe: B - R
    katalog["phot_rp_mean_mag"], # Scheinbare Helligkeit: R
    marker=".", markersize=2, linestyle="None", alpha=0.3, rasterized=True
)

# Plotting the isochrone
(g, bp, rp, label_string) = get_apparent_isochrone(init_vrot_index, init_metallicity_index, init_age_index, init_distance, init_A_V)
line, = ax.plot(bp-rp, rp, color="red", linewidth=1.0) 
text = ax.text(0.5, 0.95, label_string, color="red", horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)

# The function to be called anytime a slider or botton is changed
def update(val):
    vrot_index = vrot_button.index_selected
    metallicity_index = metallicity_slider.val
    age_index = age_slider.val
    distance = dist_slider.val
    A_V = ext_slider.val
    (g, bp, rp, label_string) = get_apparent_isochrone(vrot_index, metallicity_index, age_index, distance, A_V)
    line.set_xdata(bp-rp)
    line.set_ydata(rp)
    text.set_text(label_string)
    fig.canvas.draw_idle()

# Slider to control the distance
axdist = fig.add_axes([0.05, 0.25, 0.0225, 0.6])
dist_slider = matplotlib.widgets.Slider(
    ax=axdist,
    label=r"$d$ [pc]",
    valmin=10,
    valmax=10000,
    valinit=init_distance,
    orientation="vertical"
)

# Slider to control the age
axage = fig.add_axes([0.09, 0.25, 0.0225, 0.6])
age_slider = matplotlib.widgets.Slider(
    ax=axage,
    label=r"$i_{\mathrm{Age}}$",
    valmin=0,
    valmax=len(logages)-1,
    valstep=1,
    valinit=init_age_index,
    orientation="vertical"
)

# Slider to control the metallicity
axmetal = fig.add_axes([0.13, 0.25, 0.0225, 0.6])
metallicity_slider = matplotlib.widgets.Slider(
    ax=axmetal,
    label=r"$i_{\mathrm{[Fe/H]}}$",
    valmin=0,
    valmax=len(metallicities)-1,
    valstep=1,
    valinit=init_metallicity_index,
    orientation="vertical"
)

# Slider to control the extinction
axext = fig.add_axes([0.17, 0.25, 0.0225, 0.6])
ext_slider = matplotlib.widgets.Slider(
    ax=axext,
    label=r"$A_{\mathrm{V}}$",
    valmin=0.0,
    valmax=3.0,
    valinit=init_A_V,
    orientation="vertical"
)

# Radio buttons to control the rotation
axvrot = fig.add_axes([0.05, 0.1, 0.14, 0.08])
vrot_button = matplotlib.widgets.RadioButtons(
    ax=axvrot, 
    labels=("No rotation", r"$v/v_{\mathrm{crit}} = 0.4$"),
    active=init_vrot_index
)

# Register the update function with each slider and button:
age_slider.on_changed(update)
dist_slider.on_changed(update)
metallicity_slider.on_changed(update)
ext_slider.on_changed(update)
vrot_button.on_clicked(update)

# Finalize plot
ax.invert_yaxis()
ax.set_xlabel("$B$ - $R$")
ax.set_ylabel("$R$")
ax.set_title(f"Gaia-FHD von {sternhaufen_name}")
plt.show()